순환 신경망 모델에서 주로 사용하는 LSTM 레이어에 대해서 알아보겠습니다. 시퀀스 데이터를 다루는 레이어라 설정 파라미터에 따라 다양하게 모델을 구성할 수 있습니다. 그만큼 헷갈리는 부분도 있지만 쉬운 예제부터 차근히 살펴보겠습니다.

---

### 긴 시퀀스를 기억할 수 있는 LSTM (Long Short-Term Memory units)  레이어

케라스에서 제공하는 순환 신경망 레이어는 SimpleRNN, GRU, LSTM이 있으나 주로 사용하는 LSTM에 대해서 알아보겠습니다. 이 LSTM은 아래와 같이 간단히 사용할 수 있습니다.

    LSTM(3, input_dim=1)

주요 인자는 다음과 같습니다.
* 첫번째 인자 : 출력 뉴런의 수 입니다.
* input_dim : 입력 뉴런의 수 입니다.

이는 앞서 살펴본 Dense 레이어 형태와 비슷합니다.

    Dense(3, input_dim=1)
    
Dense와 LSTM을 레고 블록으로 도식화 하면 다음과 같습니다. 왼쪽이 Dense이고, 오른쪽이 LSTM 입니다.

![img](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_1.png)
    
사실 LSTM의 내부구조는 복잡하지만 간소화하여 외형만 표시한 것입니다. Dense 레이어와 비교한다면 히든 뉴런들이 밖으로 도출되어 있음을 보실 수 있습니다. 이러한 히든 뉴런이 재귀적으로 다시 입력으로 들어가게 되어 순환 신경망이 되는 것입니다. 간단한 예제로 하나 하나 살펴보겠습니다. 

---

### 시퀀스 데이터 준비

순환 신경망은 주로 자연어 처리에 많이 쓰이기 때문에 문장 학습 예제가 일반적이지만 본 강좌에서는 악보 학습을 해보겠습니다. 그 이유는 
- 음계가 문장보다 더 코드화 하기 쉽고, 
- 시계열 자료이며, 
- 나온 결과를 악보로 볼 수 있으며,
- 무엇보다 우리가 학습한 모델이 연주하는 곡을 들어볼 수 있기 때문입니다. 
일단 쉬운 악보인 '나비야'를 준비했습니다.

![img](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_2.png)

음표 밑에 간단한 음표코드를 표시하였습니다. 알파벳은 음계를 나타내며, 숫자는 음의 길이를 나타냅니다.
- c(도), d(레), e(미), f(파), g(솔), a(라), b(시)
- 4(4분음표), 8(8분음표)

---

### 문제 정의

"나비야"노래는 우리에게 너무나 익숙한 노래입니다. 만약 옆사람이 "나비야~ 나"까지만 불러도 나머지를 이어서 다 부를 수 있을 정도로 말이죠. 이렇게 첫 4개 음표를 입력하면 나머지를 연주할 수 있는 모델을 만드는 것이 목표입니다. 이를 위해서 다음과 같이 학습을 시켜보겠습니다. 

- 파란색 박스가 입력값이고, 빨간색 박스가 우리가 원하는 출력값입니다. 
- 1~4번째 음표를 데이터로 5번째 음표를 라벨값으로 학습을 시킵니다.
- 다음에는 2~5번째 음표를 데이터로 6번째 음표를 라벨값으로 학습을 시킵니다.
- 이후 한 음표씩 넘어가면서 노래 끝까지 학습시킵니다.

![img](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_3.png)

총 음계가 7개이고 음길이가 2개이기 때문에 구성할 수 있는 음표는 총 14개입니다. 따라서 다중 클래스 분류 문제로 풀어보도록 하겠습니다. 첫 줄 코드는 다음과 같습니다. 

* g8 e8 e4 | f8 d8 d4 | c8 d8 e8 f8 | g8 g8 g4

첫 줄에 대해 음표 4개 입력과 1개 출력을 구성하는 데이터셋은 다음과 같습니다. 1~4번째 열은 속성(feature)이고, 5번째 열은 클래스(class)를 나타냅니다.

* g8, e8, e4, f8, d8
* e8, e4, f8, d8, d4
* e4, f8, d8, d4, c8
* f8, d8, d4, c8, d8
* d8, d4, c8, d8, e8
* d4, c8, d8, e8, f8
* c8, d8, e8, f8, g8
* d8, e8, f8, g8, g8
* e8, f8, g8, g8, g4
* ...

이렇게 4개씩 구간을 보는 것을 윈도우 크기가 4라고 합니다. 그럼 다음으로 시퀀스 데이터를 윈도우 사이즈만큼 잘라서 데이터셋을 구성해보도록 하겠습니다.

---

### 데이터셋 생성

아래 seq2dataset 함수는 입력되는 시퀀스 데이터를 윈도우 크기에 따라 재구성해서 데이터셋을 생성하는 코드입니다.

In [55]:
import numpy as np

def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        dataset.append(seq[i:(i+window_size+1)])
    return np.array(dataset)

seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4', 
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

dataset = seq2dataset(seq, 4)

print(dataset.shape)
print(dataset)

(50, 5)
[['g8' 'e8' 'e4' 'f8' 'd8']
 ['e8' 'e4' 'f8' 'd8' 'd4']
 ['e4' 'f8' 'd8' 'd4' 'c8']
 ['f8' 'd8' 'd4' 'c8' 'd8']
 ['d8' 'd4' 'c8' 'd8' 'e8']
 ['d4' 'c8' 'd8' 'e8' 'f8']
 ['c8' 'd8' 'e8' 'f8' 'g8']
 ['d8' 'e8' 'f8' 'g8' 'g8']
 ['e8' 'f8' 'g8' 'g8' 'g4']
 ['f8' 'g8' 'g8' 'g4' 'g8']
 ['g8' 'g8' 'g4' 'g8' 'e8']
 ['g8' 'g4' 'g8' 'e8' 'e8']
 ['g4' 'g8' 'e8' 'e8' 'e8']
 ['g8' 'e8' 'e8' 'e8' 'f8']
 ['e8' 'e8' 'e8' 'f8' 'd8']
 ['e8' 'e8' 'f8' 'd8' 'd4']
 ['e8' 'f8' 'd8' 'd4' 'c8']
 ['f8' 'd8' 'd4' 'c8' 'e8']
 ['d8' 'd4' 'c8' 'e8' 'g8']
 ['d4' 'c8' 'e8' 'g8' 'g8']
 ['c8' 'e8' 'g8' 'g8' 'e8']
 ['e8' 'g8' 'g8' 'e8' 'e8']
 ['g8' 'g8' 'e8' 'e8' 'e4']
 ['g8' 'e8' 'e8' 'e4' 'd8']
 ['e8' 'e8' 'e4' 'd8' 'd8']
 ['e8' 'e4' 'd8' 'd8' 'd8']
 ['e4' 'd8' 'd8' 'd8' 'd8']
 ['d8' 'd8' 'd8' 'd8' 'd8']
 ['d8' 'd8' 'd8' 'd8' 'e8']
 ['d8' 'd8' 'd8' 'e8' 'f4']
 ['d8' 'd8' 'e8' 'f4' 'e8']
 ['d8' 'e8' 'f4' 'e8' 'e8']
 ['e8' 'f4' 'e8' 'e8' 'e8']
 ['f4' 'e8' 'e8' 'e8' 'e8']
 ['e8' 'e8' 'e8' 'e8' 'e8']
 ['e8' 'e8' 

4개의 속성과 1개의 클래스 값으로 구성된 샘플이 총 50개가 생성되었습니다. 모델에 데이터를 입력하기 위해서는 문자가 아닌 정수나 실수 등으로 수치를 입력해야 합니다. 각 음표에 대해서 임의의 정수를 부여하고, 이를 변환하는 자료형을 만들어보겠습니다. 추가로 seq2dataset() 함수를 수정해서, 음표 집합을 입력하면 정수 데이터셋을 반환하도록 수정하겠습니다.

In [56]:
code2int = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

int2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2int[item] for item in subset])
    return np.array(dataset)

dataset = seq2dataset(seq, 4)

print(dataset.shape)
print(dataset)

(50, 5)
[[11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  8]
 [ 8  1  7  8  9]
 [ 1  7  8  9 10]
 [ 7  8  9 10 11]
 [ 8  9 10 11 11]
 [ 9 10 11 11  4]
 [10 11 11  4 11]
 [11 11  4 11  9]
 [11  4 11  9  9]
 [ 4 11  9  9  9]
 [11  9  9  9 10]
 [ 9  9  9 10  8]
 [ 9  9 10  8  1]
 [ 9 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9 11]
 [ 1  7  9 11 11]
 [ 7  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]
 [11  9  9  2  8]
 [ 9  9  2  8  8]
 [ 9  2  8  8  8]
 [ 2  8  8  8  8]
 [ 8  8  8  8  8]
 [ 8  8  8  8  9]
 [ 8  8  8  9  3]
 [ 8  8  9  3  9]
 [ 8  9  3  9  9]
 [ 9  3  9  9  9]
 [ 3  9  9  9  9]
 [ 9  9  9  9  9]
 [ 9  9  9  9 10]
 [ 9  9  9 10  4]
 [ 9  9 10  4 11]
 [ 9 10  4 11  9]
 [10  4 11  9  2]
 [ 4 11  9  2 10]
 [11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9 11]
 [ 1  7  9 11 11]
 [ 7  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]]


---

### 다층 퍼셉트론 모델

앞서 생성한 데이터셋으로 먼저 다층 퍼셋트론 모델을 학습시켜보겠습니다. 

#### 학습

총 50개의 샘플이고, 각 샘플은 5개의 값을 가지고 있는데, 4개가 속성이고, 1개가 클래스를 나타내고 있습니다. 나올 수 있는 코드가 0에서 13이므로 14개의 클래스를 분류하는 다중 클래스 분류 문제로 풀어봅니다.

In [91]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

# 랜덤시드 고정시키기
np.random.seed(5)

# 입력(X)과 출력(Y) 변수로 분리하기
X = dataset[:,0:4]
Y = dataset[:,4]

# normalize
X = X / float(14)
# one hot encode the output variable
Y = np_utils.to_categorical(Y)

# 모델 구성하기
model = Sequential()
model.add(Dense(128, input_dim=4, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(Y.shape[1], activation='softmax'))

#88%
#model.add(Dense(128, input_dim=4, activation='relu'))
#model.add(Dense(128, activation='relu'))
#model.add(Dense(128, activation='relu'))
#model.add(Dense(Y.shape[1], activation='softmax'))


#model = Sequential()
#model.add(Dense(128, input_dim=4, activation='relu'))
#model.add(Dense(128, activation='relu'))
#model.add(Dense(Y.shape[1], activation='softmax'))


# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습시키기
model.fit(X, Y, epochs=2000, batch_size=10, verbose=2)

# 모델 평가하기
scores = model.evaluate(X, Y)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

Epoch 1/2000
0s - loss: 2.4435 - acc: 0.2200
Epoch 2/2000
0s - loss: 2.3514 - acc: 0.3400
Epoch 3/2000
0s - loss: 2.2350 - acc: 0.3400
Epoch 4/2000
0s - loss: 2.1405 - acc: 0.3400
Epoch 5/2000
0s - loss: 2.0333 - acc: 0.3400
Epoch 6/2000
0s - loss: 1.9715 - acc: 0.3400
Epoch 7/2000
0s - loss: 1.9461 - acc: 0.3400
Epoch 8/2000
0s - loss: 1.9253 - acc: 0.3400
Epoch 9/2000
0s - loss: 1.9030 - acc: 0.3400
Epoch 10/2000
0s - loss: 1.8940 - acc: 0.3400
Epoch 11/2000
0s - loss: 1.8752 - acc: 0.3400
Epoch 12/2000
0s - loss: 1.8616 - acc: 0.3400
Epoch 13/2000
0s - loss: 1.8495 - acc: 0.3400
Epoch 14/2000
0s - loss: 1.8363 - acc: 0.3400
Epoch 15/2000
0s - loss: 1.8169 - acc: 0.3400
Epoch 16/2000
0s - loss: 1.8132 - acc: 0.3400
Epoch 17/2000
0s - loss: 1.8022 - acc: 0.3400
Epoch 18/2000
0s - loss: 1.7785 - acc: 0.3600
Epoch 19/2000
0s - loss: 1.7656 - acc: 0.3800
Epoch 20/2000
0s - loss: 1.7577 - acc: 0.3800
Epoch 21/2000
0s - loss: 1.7381 - acc: 0.3800
Epoch 22/2000
0s - loss: 1.7283 - acc: 0.38

#### 테스트

테스트 과정을 먼저 설명하도록 하겠습니다. "나비야~ 나"까지 알려주면 이후 나머지를 연주하기 위해서 다음의 과정을 수행합니다.
1. t0, t1, t2, t3에 해당하는 실제 음표를 모델에 입력하면 y0 출력이 나옵니다.
2. 출력된 y0를 t4라고 가정하고, 모델에 t1, t2, t3, t4을 입력하면 y1 출력이 나옵니다.
3. 출력된 y1을 t5라고 가정하고, 모델에 t2, t3, t4, t5을 입력하면 y2 출력이 나옵니다.
4. 출력된 y2을 t6라고 가정하고, 모델에 t3, t4, t5, t6을 입력하면 y3 출력이 나옵니다.

아래 그림에서 파란 점선 부분이 모델에 의해 예측된 음표입니다. 예측된 음표를 다시 입력으로 넣어서 다음을 예측하는 식으로 50번 반복하면, 나비야 노래 끝까지 음표를 얻을 수 있습니다.

![img](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_4.png)

아래 코드는 초기 4개의 음표를 입력한 후 지정된 수 만큼 모델을 구동하여 예측값들을 생성합니다.

In [93]:
init_seq_count = 4
prediction_count = 50

seq_in = ['g8', 'e8', 'e4', 'f8']
seq_out = seq_in
seq_in = [code2int[item] for item in seq_in]

for i in range(prediction_count):
    test_x = np.array(seq_in)
    test_x = np.reshape(test_x, (1, 4)) # batch_size, feature
    prediction = model.predict(test_x)
    item = np.argmax(prediction)
    seq_out.append(int2code[item])
    seq_in.append(item)
    seq_in.pop(0)

print(seq_out)

['g8', 'e8', 'e4', 'f8', 'e8', 'e4', 'e8', 'e4', 'g8', 'g8', 'e4', 'e8', 'e4', 'g8', 'g8', 'e4', 'e8', 'e4', 'g8', 'g8', 'e4', 'e8', 'e4', 'g8', 'g8', 'e4', 'e8', 'e4', 'g8', 'g8', 'e4', 'e8', 'e4', 'g8', 'g8', 'e4', 'e8', 'e4', 'g8', 'g8', 'e4', 'e8', 'e4', 'g8', 'g8', 'e4', 'e8', 'e4', 'g8', 'g8', 'e4', 'e8', 'e4', 'g8']


In [124]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils
import numpy as np

# 코드와 인덱스 매칭 테이블 정의
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

# 시퀀스 데이터로부터 데이터셋 생성 함수
def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2int[item] for item in subset])
    return np.array(dataset)

# 랜덤시드 고정시키기
np.random.seed(5)

# "나비야" 코드
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4', 
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

# 윈도우 크기가 4로 데이터셋 생성
dataset = seq2dataset(seq, 4)

# 입력(X)과 출력(Y) 변수로 분리하기
train_X = dataset[:,0:4]
train_Y = dataset[:,4]

max_idx_value = 13

# 입력에 대해서 정규화 함
train_X = train_X / float(max_idx_value)

# 입력을 (샘플 수, 타입스텝, 특성 수)로 형태 변환
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1]))

# 인덱스로부터 다중 클래스 라벨을 위한 one-hot 인코드로 변환
train_Y = np_utils.to_categorical(train_Y)

# 모델 정의
# 모델 구성하기
model = Sequential()
model.add(Dense(128, input_dim=4, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(train_Y.shape[1], activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습시키기
model.fit(train_X, train_Y, epochs=2000, batch_size=10, verbose=2)

# 모델 평가하기
scores = model.evaluate(train_X, train_Y)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 악보 생성하기

# 최대 예측 개수 정의
prediction_count = 50

# '나비야~나'까지 입력
seq_in = ['g8', 'e8', 'e4', 'f8']
seq_out = seq_in

# 입력 코드를 인덱스화 시킨 다음 정규화 처리
seq_in = [code2idx[item] / float(max_idx_value) for item in seq_in]

for i in range(prediction_count):
    test_x = np.array(seq_in)
    test_x = np.reshape(test_x, (1, 4)) # (batch_size, timesteps, data_dim)
    
    # 4개를 입력하여 one-hot 인코딩 된 결과 얻음
    prediction_onehot = model.predict(test_x)
    
    # one-hot 인코딩을 인덱스 값으로 변환
    prediction_index = np.argmax(prediction_onehot)
    
    # seq_out는 최종 악보이므로 인덱스 값을 코드로 변환하여 저장
    seq_out.append(idx2code[prediction_index])
    
    # 다음 예측을 위해 seq_in에 신규 항목을 추가하고, 기존 항목 제거
    seq_in.append(prediction_index / float(max_idx_value))
    seq_in.pop(0)
    
print(seq_out)

Epoch 1/2000
1s - loss: 2.4410 - acc: 0.2200
Epoch 2/2000
0s - loss: 2.3440 - acc: 0.3400
Epoch 3/2000
0s - loss: 2.2236 - acc: 0.3400
Epoch 4/2000
0s - loss: 2.1279 - acc: 0.3400
Epoch 5/2000
0s - loss: 2.0232 - acc: 0.3400
Epoch 6/2000
0s - loss: 1.9661 - acc: 0.3400
Epoch 7/2000
0s - loss: 1.9426 - acc: 0.3400
Epoch 8/2000
0s - loss: 1.9204 - acc: 0.3400
Epoch 9/2000
0s - loss: 1.8986 - acc: 0.3400
Epoch 10/2000
0s - loss: 1.8898 - acc: 0.3400
Epoch 11/2000
0s - loss: 1.8708 - acc: 0.3400
Epoch 12/2000
0s - loss: 1.8563 - acc: 0.3400
Epoch 13/2000
0s - loss: 1.8454 - acc: 0.3400
Epoch 14/2000
0s - loss: 1.8324 - acc: 0.3400
Epoch 15/2000
0s - loss: 1.8119 - acc: 0.3400
Epoch 16/2000
0s - loss: 1.8077 - acc: 0.3400
Epoch 17/2000
0s - loss: 1.7962 - acc: 0.3400
Epoch 18/2000
0s - loss: 1.7719 - acc: 0.3800
Epoch 19/2000
0s - loss: 1.7586 - acc: 0.3800
Epoch 20/2000
0s - loss: 1.7509 - acc: 0.3800
Epoch 21/2000
0s - loss: 1.7316 - acc: 0.3800
Epoch 22/2000
0s - loss: 1.7209 - acc: 0.38

In [125]:
train_Y.shape[1]

12

위 모델에서 나온 결과값을 악보로 표시해보겠습니다. 박자가 안 맞는 부분은 제가 쉼표를 넣었습니다. 

![img](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_5.png)

결과는 썩 좋지가 않습니다. Dense 레이어로 구성한 MLP 모델이 작곡/연주한 것을 들어보시죠. 쉼표가 많다보니 나비가 기어가는 느낌지고, 전반적으로 우울합니다.

[음원 다운로드](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_butterfly_MLP_3_dense.mp3) 

![](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_butterfly_MLP_3_dense.mp3 http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_butterfly_MLP_3_dense.ogg)

---

### LSTM 첫번째 모델

이번에는 간단한 LSTM 모델로 학습을 시켜보겠습니다.

#### 학습

총 50개의 샘플이고, 각 샘플은 5개의 값을 가지고 있는데, 4개가 속성이고, 1개가 클래스를 나타내고 있습니다. 나올 수 있는 코드가 0에서 13이므로 14개의 클래스를 분류하는 다중 클래스 분류 문제로 풀어봅니다.

In [99]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils
import numpy as np

# 코드와 인덱스 매칭 테이블 정의
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

# 시퀀스 데이터로부터 데이터셋 생성 함수
def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2int[item] for item in subset])
    return np.array(dataset)

# 랜덤시드 고정시키기
np.random.seed(5)

# "나비야" 코드
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4', 
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

# 윈도우 크기가 4로 데이터셋 생성
dataset = seq2dataset(seq, 4)

# 입력(X)과 출력(Y) 변수로 분리하기
train_X = dataset[:,0:4]
train_Y = dataset[:,4]

# 입력에 대해서 정규화 함
train_X = train_X / float(14)

# 입력을 (샘플 수, 타입스텝, 특성 수)로 형태 변환
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))

# 인덱스로부터 다중 클래스 라벨을 위한 one-hot 인코드로 변환
train_Y = np_utils.to_categorical(train_Y)

# 모델 정의
# batch_input_shape = (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(128, batch_input_shape = (1, 4, 1), stateful=True))
model.add(Dense(train_Y.shape[1], activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

num_epoch = 300

# 모델 학습시키기
for epoch_idx in range(num_epoch):
    print ('epochs : ' + str(epoch_idx) )
    model.fit(train_X, train_Y, epochs=1, batch_size=1, verbose=2, shuffle=False) # 50 is X.shape[0]
    model.reset_states()
    
# 모델 평가하기
scores = model.evaluate(train_X, train_Y, batch_size=1, verbose=1)
model.reset_states()

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 악보 생성하기

# 최대 예측 개수 정의
prediction_count = 50

# '나비야~나'까지 입력
seq_in = ['g8', 'e8', 'e4', 'f8']
seq_out = seq_in

# 입력 코드를 인덱스화 시킨 다음 정규화 처리
seq_in = [code2idx[item] / float(14) for item in seq_in]

for i in range(prediction_count):
    test_x = np.array(seq_in)
    test_x = np.reshape(test_x, (1, 4, 1)) # (batch_size, timesteps, data_dim)
    
    # 4개를 입력하여 one-hot 인코딩 된 결과 얻음
    prediction_onehot = model.predict(test_x)
    
    # one-hot 인코딩을 인덱스 값으로 변환
    prediction_index = np.argmax(prediction_onehot)
    
    # seq_out는 최종 악보이므로 인덱스 값을 코드로 변환하여 저장
    seq_out.append(idx2code[prediction_index])
    
    # 다음 예측을 위해 seq_in에 신규 항목을 추가하고, 기존 항목 제거
    seq_in.append(prediction_index / float(14))
    seq_in.pop(0)
    
print(seq_out)

(50, 4, 1)
Epoch 1/1000
7s - loss: 2.4092 - acc: 0.2200
Epoch 2/1000
3s - loss: 2.0724 - acc: 0.3400
Epoch 3/1000
3s - loss: 1.9598 - acc: 0.3400
Epoch 4/1000
3s - loss: 1.9506 - acc: 0.3400
Epoch 5/1000
3s - loss: 1.9417 - acc: 0.3400
Epoch 6/1000
3s - loss: 1.9408 - acc: 0.3400
Epoch 7/1000
3s - loss: 1.9407 - acc: 0.3400
Epoch 8/1000
3s - loss: 1.9269 - acc: 0.3400
Epoch 9/1000
3s - loss: 1.9111 - acc: 0.3400
Epoch 10/1000
3s - loss: 1.9287 - acc: 0.3400
Epoch 11/1000
3s - loss: 1.9218 - acc: 0.3400
Epoch 12/1000
3s - loss: 1.9190 - acc: 0.3400
Epoch 13/1000
3s - loss: 1.9161 - acc: 0.3400
Epoch 14/1000
3s - loss: 1.9397 - acc: 0.3400
Epoch 15/1000
3s - loss: 1.9298 - acc: 0.3400
Epoch 16/1000
3s - loss: 1.9080 - acc: 0.3400
Epoch 17/1000
3s - loss: 1.9089 - acc: 0.3400
Epoch 18/1000
3s - loss: 1.9068 - acc: 0.3400
Epoch 19/1000
3s - loss: 1.9114 - acc: 0.3400
Epoch 20/1000
3s - loss: 1.9104 - acc: 0.3400
Epoch 21/1000
3s - loss: 1.9047 - acc: 0.3400
Epoch 22/1000
3s - loss: 1.8964 

In [122]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils
import numpy as np

# 코드와 인덱스 매칭 테이블 정의
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

# 시퀀스 데이터로부터 데이터셋 생성 함수
def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2int[item] for item in subset])
    return np.array(dataset)

# 랜덤시드 고정시키기
np.random.seed(5)

# "나비야" 코드
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4', 
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

# 윈도우 크기가 4로 데이터셋 생성
dataset = seq2dataset(seq, 4)

# 입력(X)과 출력(Y) 변수로 분리하기
train_X = dataset[:,0:4]
train_Y = dataset[:,4]

max_idx_value = 13

# 입력에 대해서 정규화 함
train_X = train_X / float(max_idx_value)

# 입력을 (샘플 수, 타입스텝, 특성 수)로 형태 변환
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))

# 인덱스로부터 다중 클래스 라벨을 위한 one-hot 인코드로 변환
train_Y = np_utils.to_categorical(train_Y)

# 모델 정의
# batch_input_shape = (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(128, batch_input_shape = (1, 4, 1)))
model.add(Dense(train_Y.shape[1], activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_X, train_Y, epochs=1000, batch_size=1, verbose=2, shuffle=False) # 50 is X.shape[0]
    
# 모델 평가하기
scores = model.evaluate(train_X, train_Y, batch_size=1, verbose=1)
model.reset_states()

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 악보 생성하기

# 최대 예측 개수 정의
prediction_count = 50

# '나비야~나'까지 입력
seq_in = ['g8', 'e8', 'e4', 'f8']
seq_out = seq_in

# 입력 코드를 인덱스화 시킨 다음 정규화 처리
seq_in = [code2idx[item] / float(max_idx_value) for item in seq_in]

for i in range(prediction_count):
    test_x = np.array(seq_in)
    test_x = np.reshape(test_x, (1, 4, 1)) # (batch_size, timesteps, data_dim)
    
    # 4개를 입력하여 one-hot 인코딩 된 결과 얻음
    prediction_onehot = model.predict(test_x)
    
    # one-hot 인코딩을 인덱스 값으로 변환
    prediction_index = np.argmax(prediction_onehot)
    
    # seq_out는 최종 악보이므로 인덱스 값을 코드로 변환하여 저장
    seq_out.append(idx2code[prediction_index])
    
    # 다음 예측을 위해 seq_in에 신규 항목을 추가하고, 기존 항목 제거
    seq_in.append(prediction_index / float(max_idx_value))
    seq_in.pop(0)
    
print(seq_out)

Epoch 1/1000
2s - loss: 2.3993 - acc: 0.2000
Epoch 2/1000
0s - loss: 2.0469 - acc: 0.3400
Epoch 3/1000
0s - loss: 1.9598 - acc: 0.3400
Epoch 4/1000
0s - loss: 1.9412 - acc: 0.3400
Epoch 5/1000
0s - loss: 1.9330 - acc: 0.3400
Epoch 6/1000
0s - loss: 1.9280 - acc: 0.3400
Epoch 7/1000
0s - loss: 1.9243 - acc: 0.3400
Epoch 8/1000
0s - loss: 1.9214 - acc: 0.3400
Epoch 9/1000
0s - loss: 1.9190 - acc: 0.3400
Epoch 10/1000
0s - loss: 1.9169 - acc: 0.3400
Epoch 11/1000
0s - loss: 1.9150 - acc: 0.3400
Epoch 12/1000
0s - loss: 1.9133 - acc: 0.3400
Epoch 13/1000
0s - loss: 1.9118 - acc: 0.3400
Epoch 14/1000
0s - loss: 1.9103 - acc: 0.3400
Epoch 15/1000
0s - loss: 1.9089 - acc: 0.3400
Epoch 16/1000
0s - loss: 1.9075 - acc: 0.3400
Epoch 17/1000
0s - loss: 1.9061 - acc: 0.3400
Epoch 18/1000
0s - loss: 1.9047 - acc: 0.3400
Epoch 19/1000
0s - loss: 1.9033 - acc: 0.3400
Epoch 20/1000
0s - loss: 1.9018 - acc: 0.3400
Epoch 21/1000
0s - loss: 1.9002 - acc: 0.3400
Epoch 22/1000
0s - loss: 1.8984 - acc: 0.34

케라스 버전

In [116]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils
import numpy as np

# 코드와 인덱스 매칭 테이블 정의
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

# 시퀀스 데이터로부터 데이터셋 생성 함수
def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2int[item] for item in subset])
    return np.array(dataset)

# 랜덤시드 고정시키기
np.random.seed(5)

# "나비야" 코드
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4', 
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

# 윈도우 크기가 4로 데이터셋 생성
dataset = seq2dataset(seq, 4)

# 입력(X)과 출력(Y) 변수로 분리하기
train_X = dataset[:,0:4]
train_Y = dataset[:,4]

max_idx_value = 13

# 입력에 대해서 정규화 함
train_X = train_X / float(max_idx_value)

# 입력을 (샘플 수, 타입스텝, 특성 수)로 형태 변환
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))

# 인덱스로부터 다중 클래스 라벨을 위한 one-hot 인코드로 변환
train_Y = np_utils.to_categorical(train_Y)

# 모델 정의
# batch_input_shape = (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(128, batch_input_shape = (1, 4, 1), stateful=True))
model.add(Dense(train_Y.shape[1], activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

num_epoch = 300

# 모델 학습시키기
for epoch_idx in range(num_epoch):
    print ('epochs : ' + str(epoch_idx) )
    model.fit(train_X, train_Y, epochs=1, batch_size=1, verbose=2, shuffle=False) # 50 is X.shape[0]
    model.reset_states()
    
# 모델 평가하기
scores = model.evaluate(train_X, train_Y, batch_size=1, verbose=1)
model.reset_states()

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 악보 생성하기

# 최대 예측 개수 정의
prediction_count = 50

# '나비야~나'까지 입력
seq_in = ['g8', 'e8', 'e4', 'f8']
seq_out = seq_in

# 입력 코드를 인덱스화 시킨 다음 정규화 처리
seq_in = [code2idx[item] / float(max_idx_value) for item in seq_in]

for i in range(prediction_count):
    test_x = np.array(seq_in)
    test_x = np.reshape(test_x, (1, 4, 1)) # (batch_size, timesteps, data_dim)
    
    # 4개를 입력하여 one-hot 인코딩 된 결과 얻음
    prediction_onehot = model.predict(test_x)
    
    # one-hot 인코딩을 인덱스 값으로 변환
    prediction_index = np.argmax(prediction_onehot)
    
    # seq_out는 최종 악보이므로 인덱스 값을 코드로 변환하여 저장
    seq_out.append(idx2code[prediction_index])
    
    # 다음 예측을 위해 seq_in에 신규 항목을 추가하고, 기존 항목 제거
    seq_in.append(prediction_index / float(max_idx_value))
    seq_in.pop(0)
    
print(seq_out)

epochs : 0
Epoch 1/1
2s - loss: 2.3483 - acc: 0.1400
epochs : 1
Epoch 1/1
0s - loss: 2.0417 - acc: 0.3400
epochs : 2
Epoch 1/1
0s - loss: 1.9635 - acc: 0.3400
epochs : 3
Epoch 1/1
0s - loss: 1.9469 - acc: 0.3400
epochs : 4
Epoch 1/1
0s - loss: 1.9369 - acc: 0.3400
epochs : 5
Epoch 1/1
0s - loss: 1.9301 - acc: 0.3400
epochs : 6
Epoch 1/1
0s - loss: 1.9248 - acc: 0.3600
epochs : 7
Epoch 1/1
0s - loss: 1.9205 - acc: 0.3600
epochs : 8
Epoch 1/1
0s - loss: 1.9168 - acc: 0.3600
epochs : 9
Epoch 1/1
0s - loss: 1.9134 - acc: 0.3600
epochs : 10
Epoch 1/1
0s - loss: 1.9102 - acc: 0.3600
epochs : 11
Epoch 1/1
0s - loss: 1.9070 - acc: 0.3600
epochs : 12
Epoch 1/1
0s - loss: 1.9036 - acc: 0.3600
epochs : 13
Epoch 1/1
0s - loss: 1.8997 - acc: 0.3600
epochs : 14
Epoch 1/1
0s - loss: 1.8954 - acc: 0.3600
epochs : 15
Epoch 1/1
0s - loss: 1.9070 - acc: 0.3600
epochs : 16
Epoch 1/1
0s - loss: 1.8938 - acc: 0.3600
epochs : 17
Epoch 1/1
0s - loss: 1.8822 - acc: 0.3600
epochs : 18
Epoch 1/1
0s - loss: 2.062

['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4', 'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4', 'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']


In [109]:
seq_in

[0.7857142857142857,
 0.6428571428571429,
 0.14285714285714285,
 0.7142857142857143]

In [ ]:





# as the first layer in a Sequential model
model = Sequential()
model.add(LSTM(32, input_shape=(10, 64)`))
# now model.output_shape == (None, 32)
# note: `None` is the batch dimension.

# for subsequent layers, no need to specify the input size:
model.add(LSTM(16))

# to stack recurrent layers, you must use return_sequences=True
# on any recurrent layer that feeds into another recurrent layer.
# note that you only need to specify the input size on the first layer.
model = Sequential()
model.add(LSTM(64, input_dim=64, input_length=10, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(10))

---

### 결론

본 강좌를 통해 컨볼루션 모델에서 사용되는 주요 레이어에 대해서 알아보았고, 레이어를 조합하여 컨볼루션 모델을 만들어봤습니다. 다음 강좌에서는 이 모델을 이용하여 실제로 데이터셋을 학습시킨 후 분류가 제대로 되는 지 알아보겠습니다.

---

### 같이 보기

* [강좌 목차](https://tykimos.github.io/Keras/lecture/)
* 이전 : [딥러닝 이야기/다층 퍼셉트론 모델 만들어보기](https://tykimos.github.io/Keras/2017/02/04/MLP_Getting_Started/)
* 다음 : [딥러닝 이야기/컨볼루션 신경망 모델 만들어보기](https://tykimos.github.io/Keras/2017/03/08/CNN_Getting_Started/)

In [ ]:
\paper { 
  indent = 0\mm
}

\header{
  title = "나 비 야"
}

melody = \relative c'' {
\clef treble
\key c \major
\autoBeamOff
\time 2/4
g8 e8 e4 
f8 d8 d4 
c8 d8 e8 f8 
g8 g8 g4
\break
g8 e8 e8 e8 
f8 d8 d4
c8 e8 g8 g8 
e8 e8 e4
\break
d8 d8 d8 d8 
d8 e8 f4 
e8 e8 e8 e8 
e8 f8 g4
\break
g8 e8 e4 
f8 d8 d4 
c8 e8 g8 g8 
e8 e8 e4
}

\addlyrics {
#"g8" #"e8" #"e4"
#"f8" #"d8" #"d4"
#"c8" #"d8" #"e8" #"f8"
#"g8" #"g8" #"g4"
#"g8" #"e8" #"e8" #"e8" 
#"f8" #"d8" #"d4"
#"c8" #"e8" #"g8" #"g8"
#"e8" #"e8" #"e4"
#"d8" #"d8" #"d8" #"d8"
#"d8" #"e8" #"f4"
#"e8" #"e8" #"e8" #"e8"
#"e8" #"f8" #"g4"
#"g8" #"e8" #"e4" 
#"f8" #"d8" #"d4" 
#"c8" #"e8" #"g8" #"g8"
#"e8" #"e8" #"e4"
}

\score {
  \new Staff \melody
  \layout { }
  \midi { }
}

\version "2.18.2"  % necessary for upgrading to future LilyPond versions.
